In [1]:
from vllm import LLM, SamplingParams
import torch
torch.cuda.empty_cache()

INFO 06-06 13:03:02 [__init__.py:243] Automatically detected platform cuda.


In [2]:
llm = LLM(
    model="../llama-3-8b",
    max_logprobs=100
    )

INFO 06-06 13:03:03 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-06 13:03:03 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-06 13:03:03 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-06 13:03:07 [config.py:793] This model supports multiple tasks: {'generate', 'embed', 'reward', 'classify', 'score'}. Defaulting to 'generate'.
INFO 06-06 13:03:07 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-06 13:03:08 [core.py:438] Waiting for init message from front-end.
INFO 06-06 13:03:08 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='../llama-3-8b', speculative_config=None, tokenizer='../llama-3-8b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 06-06 13:03:10 [default_loader.py:280] Loading weights took 1.80 seconds
INFO 06-06 13:03:11 [gpu_model_runner.py:1549] Model loading took 14.9596 GiB and 1.892042 seconds
INFO 06-06 13:03:14 [backends.py:459] Using cache directory: /home/ispahan/.cache/vllm/torch_compile_cache/d5073315d8/rank_0_0 for vLLM's torch.compile
INFO 06-06 13:03:14 [backends.py:469] Dynamo bytecode transform time: 3.11 s
INFO 06-06 13:03:15 [backends.py:158] Cache the graph of shape None for later use
INFO 06-06 13:03:25 [backends.py:170] Compiling a graph for general shape takes 11.48 s
INFO 06-06 13:03:34 [monitor.py:33] torch.compile takes 14.59 s in total
INFO 06-06 13:03:35 [kv_cache_utils.py:637] GPU KV cache size: 36,784 tokens
INFO 06-06 13:03:35 [kv_cache_utils.py:640] Maximum concurrency for 8,192 tokens per request: 4.49x
INFO 06-06 13:03:54 [gpu_model_runner.py:1933] Graph capturing finished in 19 secs, took 0.51 GiB
INFO 06-06 13:03:54 [core.py:167] init engine (profile, create kv cache, war

In [3]:
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=256,
    logprobs=30  # Return top-5 token log probs at each step
)

In [4]:
prompt = "The capital of France is"
outputs = llm.generate([prompt], sampling_params)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|               | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks…

In [5]:
output = outputs[0]
generated_text = output.outputs[0].text
token_logprobs = output.outputs[0].logprobs

print(f"Generated: {generated_text}")

# Correct way to access log probs
for i, token_data in enumerate(token_logprobs):
    print(f"Position {i}:")
    if i >= 30:
        break
    for token_id, logprob_obj in token_data.items():
        token = llm.get_tokenizer().decode([token_id])
        print(f"  '{token}': {logprob_obj.logprob:.4f}")

Generated:  a big city with a lot of history, culture, and architecture. Paris has a lot of different neighborhoods, each with its own unique character. The city is home to many famous landmarks, such as the Eiffel Tower, the Louvre Museum, and the Notre Dame Cathedral.
There are also a lot of great restaurants, cafes, and shops in Paris. The city is a great place to explore on foot or by bike. There are also many parks and gardens in Paris, making it a great place to relax and enjoy the outdoors.
Paris is a beautiful city that is full of culture and history. It is a great place to explore and learn about different cultures. There are many different neighborhoods in Paris, each with its own unique character. Paris is a great place to visit and explore.
1 What are the 4 main parts of Paris?
2 What are the 3 main areas of Paris?
3 What are the 12 arrondissements of Paris?
4 What is the most dangerous part of Paris?
5 What are the 5 main districts in Paris?
6 What is the best area to stay

In [ ]:
output = outputs[0]
generated_text = output.outputs[0].text
token_logprobs = output.outputs[0].logprobs

print(f"Generated: {generated_text}")

# Correct way to access log probs
for token_data in output.outputs[0].logprobs[:10]:
    print(f"Token: {token_data}")
    for token_id, logprob_obj in token_data.items():
        token = llm.get_tokenizer().decode([token_id])
        print(f"  '{token}': {logprob_obj.logprob:.4f}")